# AHE DATA ANALYSIS
This notebook finds the coercive field values for AHE measurements and will perform loopshift analysis to find the linear fit of average coercivity vs applied current.  Further input of proper device characteristics will provide estimations of effective field per current density and the spin hall angle.

Select file input and device parameters below

In [ ]:
# user inputs
dir_path = r'C:\Users\Neuromancer\Desktop\test' # path to directory
file_type = '750'
normalize_data = True # change to false to see regular y data, default is true

# Device characteristics
w = 10e-6 # device width (meters)
d = 4e-9 # thickness of spin Hall material (meters) 
t = 1.4e-9 # thickness of magnetic layer - dead layer (meters)
M = 1500 * 1000 # saturization magnetization (A/m)
rho_FM = 40 # resistivity of magnetic layer (uOhm-cm)
rho_HM = 300 # resistivity of spin Hall material (uOhm-cm)

In [ ]:
import pandas as pd
import glob
import os
import math
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from scipy import optimize
from AnalysisFunctions import drop_regions, find_zeros, import_datasets

all_files = glob.glob(os.path.join(dir_path, '*'+file_type+'*.csv')) # use os.path.join to make os independent
[all_files.remove(x) for x in all_files if 'results.csv' in x] # ignore results file

if len(all_files) != 0:
    full_df = import_datasets(all_files, normalize_data) # if True, data is automatically normalized
    display(full_df.head())
else:
    print(f'No csv files found in {dir_path} with the format: {file_type}!')

#### SELECT DATAFRAME AND GRAPH PARAMETERS:
*x_column* is the column used for x values for graphing and data analysis <br>
*y_column* is the column used for data analysis and graphing <br>
*hue_column* is the column used for seperating the data sets into individual line/scatter plots <br>
*graph_column* is the column that a set of hue_column values is grouped by <br>

The following cell will provide a list of the unique values found in the hue and graph column.

In [ ]:
x_column = 'Field(Oe)' # plot x data values
y_column = 'Normalized Voltage(mV)' # plot y data values
hue_column = 'Applied current (mA)' # column to determine how lines should be colored
graph_column = 'Applied in-plane field (Oe)' # seperates data by this column to graph plots (multi-hued)

full_df[hue_column] = full_df[hue_column].apply(lambda x: round(x, 2)) # round to two decimal places
full_df[graph_column] = full_df[graph_column].apply(lambda x: round(x, 2))

# sorted lists of unique values
hue_column_list = np.sort(full_df[hue_column].unique())
graph_column_list = np.sort(full_df[graph_column].unique())
# print values so they are easy to paste into the ignore function
print(hue_column, 'list values in the dataframe: ' + np.array2string(hue_column_list, separator=','))
print(graph_column, 'list values in the dataframe: ' + np.array2string(graph_column_list, separator=','))

In [ ]:
# plot data
sns.set_style('whitegrid')
sns.set_palette('bright', len(hue_column_list))
f = sns.FacetGrid(full_df, hue=hue_column, col=graph_column, height=7, despine=False)
f.map(plt.plot, x_column, y_column).add_legend()

#### IGNORING DATA

Should the user wish to ignore any datasets in the following analysis, including the *graph_column*  value : *hue_column* value in the *ignore_dict* in the following cell will drop the said data from the analysis.  The *graph column* should be the key string and the *hue_column* values should be in a list as ints or floats.  To ignore an *graph_column* value, use *hue_column_list* to ignore all data for said value.

In [ ]:
# fill in dictionary with values to be ignored in the subsequent data processing
ignore_dict = {
    # use format of dictionary key == graph_column value with values == list(hue_column values to ignore)
    '400.0': [-0.7, -0.3, 0.5],
}
# make copy so original import does not need to be repeated
cleaned_df = full_df.copy()
# drop the areas from the full dataframe specified in the ignore_dict
cleaned_df, ig_df, not_found = drop_regions(
    cleaned_df, ignore_dict, graph_column_list, graph_column, hue_column)
# update the hue and graph columns
hue_column_list = np.sort(cleaned_df[hue_column].unique())
graph_column_list = np.sort(cleaned_df[graph_column].unique())

for string in not_found:
    print(string)

# if there was ignored data, plot said data
if type(ig_df) == pd.DataFrame:
    f = sns.FacetGrid(ig_df, hue=hue_column,
                      col=graph_column, height=7, despine=False)
    f.map(plt.plot, x_column, y_column).add_legend()

#### DATA ANALYSIS:

AHE Data analysis checks for the coercivity values closest to the middle of the loop with the lowest index.  By changing the value of *check_left_right* the user can change how many points before and after a zero intercept should be of the same sign.  Loops that have more than two zeros will automatically be flagged and the user will be prompted whether or not to ignore those loops.

In [ ]:
# number of datapoints to compare
check_left_right = 1

# building the df to the proper size optimizes speed, will be larger than needed if data is ignored
coercivity_df = (pd.DataFrame(index=np.arange(len(graph_column_list) * len(hue_column_list)),
                              columns=['Positive X Coercivity',
                                       'Negative X Coercivity',
                                       'Average Coercivity',
                                       'Y Zeros',
                                       '3+ Zeros',
                                       hue_column,
                                       graph_column],
                              dtype='float')
                 )

# go through cleaned_df and find coercivity values (x data) and save in coercivity_df
len_index = 0
multi_zero = {}
for g in graph_column_list:
    # h_list is the specific hue values for each graph value, i.e. skips all values that are ignored
    h_list = np.sort(
        cleaned_df[(cleaned_df[graph_column] == g)][hue_column].unique())
    for h in h_list:
        # from dataframe of of specific graph_column and hue_column values pass xy values as a numpy array
        z, flag = find_zeros(cleaned_df[(cleaned_df[graph_column] == g) & (cleaned_df[hue_column] == h)]
                             .loc[:, [x_column, y_column]].to_numpy(),
                             check_left_right)

        if flag == 'True':
            if str(g) in multi_zero:
                multi_zero[str(g)].append(h)
            else:
                multi_zero[str(g)] = [h]
        if len(z) != 2:
            print(
                f'Coercivity values not found for dataset {hue_column} {h} with {graph_column} {g}')
        else:
            coercivity_df.loc[len_index] = z + \
                [(z[0] + z[1]) / 2, 0.0, flag, h, g]
        len_index += 1

if len(multi_zero) != 0:
    for key, value in multi_zero.items():
        print(f'Multiple zeros found for loop: {key} {value}')
    q = input('Do you wish to ingore data with multiple zeros? (y/n)')
    if q == 'Y' or q == 'y':
        cleaned_df, ig, nf = drop_regions(
            cleaned_df, multi_zero, graph_column_list, graph_column, hue_column)
        coercivity_df, ig, nf = drop_regions(
            coercivity_df, multi_zero, graph_column_list, graph_column, hue_column)
        hue_column_list = np.sort(cleaned_df[hue_column].unique())
        graph_column_list = np.sort(cleaned_df[graph_column].unique())
    else:
        pass
else:
    print('Only two zeros detected per loop')

coercivity_df.head()

In [ ]:
# plot data with found points
f = sns.FacetGrid(cleaned_df, hue=hue_column,
                  col=graph_column, height=7, despine=False)
f.map(plt.plot, x_column, y_column, zorder=0).add_legend()
for index, ax in enumerate(f.axes[0]):
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:, 0].to_numpy(),
               coercivity_df[(coercivity_df[graph_column] ==
                              graph_column_list[index])].iloc[:, 3].to_numpy(),
               c='black', marker='D', zorder=1)
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:, 1].to_numpy(),
               coercivity_df[(coercivity_df[graph_column] ==
                              graph_column_list[index])].iloc[:, 3].to_numpy(),
               c='black', marker='D', zorder=1)

#### AHE RESULTS
Slope, intercept, Effective Field per Current and Spin Hall Angle are found for each unique value of the *graph_column* and stored in a DataFrame.  Regardless of the device specific parameters determined by the user in the first cell, the fitting slope and intercept for each graph should be accurate to the dataset. The user has the option to ignore any specific data when saving the DataFrame in the last cell

In [ ]:
# dataframe containing data that will be saved to final output
results_df = (pd.DataFrame(index=np.arange(len(graph_column_list)),
                           columns=['Fitting Slope',
                                    'Fitting Intercept',
                                    'Effective Field Per Current (Oe/A*m^-2)',
                                    'Spin Hall Angle',
                                    graph_column])
              )

# use scipy fitting useing non-linear least squares fit


def linear_test_function(x, m, b):
    return (m * x) + b


data_index = 0
for g in graph_column_list:
    try:
        params, params_covariance = (optimize.curve_fit(linear_test_function,  # function to test
                                                        coercivity_df[(
                                                            coercivity_df[graph_column] == g)][hue_column].to_numpy(),  # x values
                                                        coercivity_df[(coercivity_df[graph_column] == g)]['Average Coercivity'].to_numpy())  # y values
                                     )
        # Current distribution ratio
        ratio = (rho_FM * d) / (rho_FM * d + rho_HM * t)
        # Calculations of switching currents/SOT efficiency/etc.
        HperJ = (params[0] * 10e11 * (1 / 1000) * (w * d)) / \
            ratio  # H/J in Oe/A*m^-2 * 10e11
        she = (2 * M * t * w * d * params[0]) / \
            (10 * 6.6e-16) * (2 / math.pi) / ratio
        results_df.loc[data_index] = [params[0], params[1], HperJ, she, g]
        data_index += 1
    except:
        print(f"An error occured with {g} {graph_column}")

results_df.head()

In [ ]:
# plot data with found coercivities and fitting lines
f = sns.FacetGrid(coercivity_df, col=graph_column, height=7, despine=False)
f.map(plt.scatter, hue_column, 'Average Coercivity')
for index, ax in enumerate(f.axes[0]):
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].loc[:, hue_column].to_numpy(),
               coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])
                             ].loc[:, 'Positive X Coercivity'].to_numpy(),
               color='green')
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].loc[:, hue_column].to_numpy(),
               coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])
                             ].loc[:, 'Negative X Coercivity'].to_numpy(),
               color='orange')
    ax.plot(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].loc[:, hue_column],
            linear_test_function(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].loc[:, hue_column].to_numpy(),
                                 *[
                results_df[(results_df[graph_column] == graph_column_list[index])
                           ]['Fitting Slope'].to_numpy()[0],  # slope per graph value
                results_df[(results_df[graph_column] == graph_column_list[index])
                           ]['Fitting Intercept'].to_numpy()[0]  # intercept per graph value
            ]))    

In [ ]:
filename = 'Loopshift'
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
save_ignore = [
    # list of values to ignore go here
    # 100,
    # -1000,
]

for n in save_ignore:
    try:
        results_df = results_df.drop(
            results_df[(results_df[graph_column] == n)].index, inplace=True)
        print(f'Dropped value {n} from the results dataframe')
    except:
        print(f'Failed to drop value {n}!')

try:
    results_df.to_csv(os.path.join(dir_path, file_type + filename +
                                   timestamp + 'results.csv'), encoding='utf-8', index=False)
    print(os.path.join(dir_path, file_type + filename +
                       timestamp + 'results.csv') + ' saved successfully')
except:
    print('Failed to save results.')